In [ ]:
# using functions in spark

In [ ]:
from pyspark.sql import SparkSession, SQLContext

In [2]:
spark = SparkSession.\
        builder.\
        config("spark.ui.port", "0").\
        config("spark.sql.warehouse.dir", "/user/band/warehouse").\
        enableHiveSupport().\
        appName("Functions Spark SQL").\
        master("yarn").\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-12-23 08:51:31,738 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark.sql("SHOW DATABASES").show()

+-----------+
|  namespace|
+-----------+
|band_retail|
|    default|
|  ordertest|
|  retail_db|
+-----------+



In [4]:
spark.sql("CREATE DATABASE IF NOT EXISTS testFunction")

2021-12-23 09:01:20,366 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
2021-12-23 09:01:20,415 WARN metastore.ObjectStore: Failed to get database testfunction, returning NoSuchObjectException


""


In [6]:
spark.sql("DROP DATABASE IF EXISTS testfunction")

""


In [12]:
spark.sql("SHOW FUNCTIONS").show(30, False)

+---------------------+
|function             |
+---------------------+
|!                    |
|!=                   |
|%                    |
|&                    |
|*                    |
|+                    |
|-                    |
|/                    |
|<                    |
|<=                   |
|<=>                  |
|<>                   |
|=                    |
|==                   |
|>                    |
|>=                   |
|^                    |
|abs                  |
|acos                 |
|acosh                |
|add_months           |
|aggregate            |
|and                  |
|any                  |
|approx_count_distinct|
|approx_percentile    |
|array                |
|array_contains       |
|array_distinct       |
|array_except         |
+---------------------+
only showing top 30 rows



In [33]:
spark.sql("DESCRIBE FUNCTION substr").show(300, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|function_desc                                                                                                                                                                                                                                                                                                                                                                              |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
spark.sql("SELECT CURRENT_DATE").show()

+--------------+
|current_date()|
+--------------+
|    2021-12-23|
+--------------+



In [36]:
from pyspark.sql import Catalog

In [48]:
spark.sql("USE retail_db").show()

++
||
++
++



In [49]:
spark.sql("SHOW TABLES")

namespace,tableName,isTemporary
retail_db,orders,false


In [50]:
spark.sql("DESCRIBE orders")

col_name,data_type,comment
order_id,int,null
order_date,string,null
order_customer_id,int,null
order_status,string,null


In [56]:
data = spark.sql("SELECT * FROM orders LIMIT 15")

In [58]:
print(type(data))

<class 'pyspark.sql.dataframe.DataFrame'>


In [62]:
data.write.format("csv").option("header", True).option("sep", ",").save("/user/band/warehouse/15_first.csv")

In [79]:
orders = spark.read.option("header", True).schema("order_id INT, order_date STRING, order_customer_id INT, order_status STRING").csv("/user/band/warehouse/15_first.csv")

In [80]:
print(orders)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [81]:
orders.createOrReplaceTempView("orders_temp")

In [82]:
spark.sql("DESCRIBE orders_temp")

col_name,data_type,comment
order_id,int,null
order_date,string,null
order_customer_id,int,null
order_status,string,null


In [84]:
spark.sql("SELECT * FROM orders_temp").show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [88]:
spark.sql("""
    SELECT order_id, order_date, order_customer_id,
    lower(order_status) AS order_status,
    length(order_status) AS order_status_length
    FROM orders LIMIT 10
""").show(truncate=True)

+--------+--------------------+-----------------+---------------+-------------------+
|order_id|          order_date|order_customer_id|   order_status|order_status_length|
+--------+--------------------+-----------------+---------------+-------------------+
|   34565|2014-02-23 00:00:...|             8702|       complete|                  8|
|   34566|2014-02-23 00:00:...|             3066|pending_payment|                 15|
|   34567|2014-02-23 00:00:...|             7314|suspected_fraud|                 15|
|   34568|2014-02-23 00:00:...|             1271|       complete|                  8|
|   34569|2014-02-23 00:00:...|            11083|       complete|                  8|
|   34570|2014-02-23 00:00:...|             3159|         closed|                  6|
|   34571|2014-02-23 00:00:...|             4551|         closed|                  6|
|   34572|2014-02-23 00:00:...|             8135|        pending|                  7|
|   34573|2014-02-23 00:00:...|             7497|pendi

In [89]:
spark.sql("""
    SELECT concat(year, '-', lpad(month, 2, 0), '-',
              lpad(myDate, 2, 0)) AS order_date
    FROM
    (SELECT 2013 AS year, 7 AS month, 25 AS myDate) q
""")

order_date
2013-07-25


In [90]:
spark.sql("""
    SELECT concat_ws('-', year, lpad(month, 2, 0),
              lpad(myDate, 2, 0)) AS order_date
    FROM
    (SELECT 2013 AS year, 7 AS month, 25 AS myDate) q
""")

order_date
2013-07-25


In [91]:
spark.sql("SELECT explode(split('2013-07-25', '-')) AS result").show()

+------+
|result|
+------+
|  2013|
|    07|
|    25|
+------+



In [99]:
spark.sql("""
        SELECT to_unix_timestamp('2019-04-30 18:18:51') AS unixtime
""").show()

+----------+
|  unixtime|
+----------+
|1556623131|
+----------+



In [101]:
spark.sql("""
    SELECT
    round(10.58) rnd,
    floor(10.58) flr,
    ceil(10.58) cl
""").show()

+---+---+---+
|rnd|flr| cl|
+---+---+---+
| 11| 10| 11|
+---+---+---+



In [102]:
spark.sql("SELECT abs(-10.5), abs(10)")

abs(-10.5),abs(10)
10.5,10


In [118]:
spark.sql("SELECT cast(round(rand() * 1) AS int) AS random_int").show()

+----------+
|random_int|
+----------+
|         1|
+----------+



In [123]:
spark.sql("DESCRIBE FUNCTION MIN").show(truncate=False)

+--------------------------------------------------------------+
|function_desc                                                 |
+--------------------------------------------------------------+
|Function: min                                                 |
|Class: org.apache.spark.sql.catalyst.expressions.aggregate.Min|
|Usage: min(expr) - Returns the minimum value of `expr`.       |
+--------------------------------------------------------------+



In [136]:
spark.sql("DESCRIBE orders").show(truncate=False)

+-----------------+---------+-------+
|col_name         |data_type|comment|
+-----------------+---------+-------+
|order_id         |int      |null   |
|order_date       |string   |null   |
|order_customer_id|int      |null   |
|order_status     |string   |null   |
+-----------------+---------+-------+



In [148]:
spark.sql("SELECT * FROM VALUES (2),3,4 AS tab(title)").show()

+-----+
|title|
+-----+
|    2|
|    3|
|    4|
+-----+



In [175]:
spark.sql("SELECT order_customer_id FROM orders LIMIT 10").show()

+-----------------+
|order_customer_id|
+-----------------+
|             8702|
|             3066|
|             7314|
|             1271|
|            11083|
|             3159|
|             4551|
|             8135|
|             7497|
|             1868|
+-----------------+



In [178]:
spark.sql("SELECT MIN(tb_min) FROM (SELECT order_customer_id AS tb_min FROM orders LIMIT 10)").show()

+-----------+
|min(tb_min)|
+-----------+
|       1271|
+-----------+

